<a href="https://colab.research.google.com/github/kcj4800/python-instruction/blob/main/%EA%B0%9C_%EA%B3%A0%EC%96%91%EC%9D%B4_%EA%B5%AC%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 808M/814M [00:04<00:00, 129MB/s]
100% 814M/814M [00:04<00:00, 171MB/s]


In [ ]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .
!unzip -q train.zip -d .

In [ ]:
import os
import tensorflow as tf
import shutil

print(len(os.listdir('/content/train/')))

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/cat/'+i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/dog/'+i)

# tf.keras.preprocessing.image_dataset_from_directory()

25000


In [ ]:
%rm -rf dataset

In [ ]:
# validation : 확인, (동) 허가하다.
# split : 쪼개다, 분해하다.
# batch : 한 묶음, 집단
# subset : 부분집합

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234
)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)

# 데이터 전처리 0 ~ 255 => 0 ~ 1 압축 => 연산속도가 빨라지고 최적의 w값도 빨리 찾기 위해서 한다. 

def 전처리함수(i, 정답): # 이미지데이터, 정답
  i = tf.cast(i/255.0, tf.float32)
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = train_ds.map(전처리함수)

# train_ds = train_ds.map(함수) : train_ds 데이터에 전부 함수를 적용시킨 뒤 리턴해준다.

# import matplotlib.pyplot as plt

for i, 정답 in train_ds.take(1):
  print(i)
  print(정답)
  # plt.imshow(i[0].numpy().astype('unit8'))
  # plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
tf.Tensor(
[[[[0.08425054 0.02934857 0.02542701]
   [0.0788603  0.02395833 0.02003676]
   [0.09209559 0.04895834 0.03327206]
   ...
   [0.06710516 0.02360026 0.00803654]
   [0.07450981 0.01960784 0.01568628]
   [0.00392923 0.00803654 0.00392923]]

  [[0.1010972  0.0501168  0.02658739]
   [0.119267   0.06828661 0.0447572 ]
   [0.06590265 0.03845167 0.01492226]
   ...
   [0.07071079 0.02720588 0.01164216]
   [0.06977443 0.01487247 0.0109509 ]
   [0.00349839 0.01146408 0.00509153]]

  [[0.09571078 0.04473039 0.01335784]
   [0.09761029 0.0466299  0.01525735]
   [0.07900965 0.066142   0.03922909]
   ...
   [0.07178692 0.02703546 0.01188726]
   [0.0753753  0.02147863 0.01722197]
   [0.00857843 0.01

In [ ]:
model = tf.keras.Sequential([
    
    # 전처리를 해주는 레이어
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64, 64, 3)), # 반반 확률로 가로로 뒤집어 준다.
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1), # 이미지를 랜덤하게 돌려준다.
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1), # 이미지를 랜덤하게 줌 해준다.

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2), # overfitting 완화기능 : 윗 레이어의 노드 중 20%를 제거해주세요.
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=3)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_rotation_1 (RandomRo  (None, 64, 64, 3)        0         
 tation)                                                         
                                                                 
 random_zoom_1 (RandomZoom)  (None, 64, 64, 3)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)       

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 전처리

생성기 = ImageDataGenerator(
    rescale=1./255,         # 255로 나눠준다.
    rotation_range=20,      # 회전
    zoom_range=0.15,        # 확대
    width_shift_range=0.2,  # 넓이 이동
    height_shift_range=0.2, # 높이 이동
    shear_range=0.15,       # 굴절
    horizontal_flip=True,    # 가로반전
    fill_mode='nearest')

트레이닝용 = 생성기.flow_from_directory(
    '/content/dataset',
    class_mode='binary',  # 두개면 binary, 몇개더면 categorical
    shuffle=True,         # 데이터를 뽑을 때 셔플여부
    seed=123,             #
    color_mode='rgb',     # 색상이 있는지, 흑백인지
    batch_size=64,        # 배치사이즈
    target_size=(64,64),  # 타겟사이즈
)

# validation_data() 에 들어갈 검증용 데이터는 굳이 뒤틀거나 할 필요가 없으므로 나눠주기만 한다.
생성기2 = ImageDataGenerator(rescale=1./255)

# validation_data() 테스트(검증용) 데이터도 똑같이 해서 검증해준다.
검증용 = 생성기2.flow_from_directory(
    '/content/dataset',
    class_mode='binary',
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
)

model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2), # overfitting 완화기능 : 윗 레이어의 노드 중 20%를 제거해주세요.
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(트레이닝용, 
          validation_data=(검증용), 
          # steps_per_epoch=20000//64,
          epochs=3)



Found 25000 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 16, 16, 64)        0         
                                  

InvalidArgumentError: ignored